In [ ]:

import os
import sys

# Pfad zum übergeordneten Verzeichnis ermitteln (einen Ordner zurück)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Das übergeordnete Verzeichnis zum Suchpfad hinzufügen
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

print(f"Das Verzeichnis '{parent_dir}' wurde zum sys.path hinzugefügt.")
#from pathlib import Path
from kg_manager_new import KGManager
# 1. Instanziieren
kg_manager = KGManager(ttl_path=r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test_filled.ttl", debug=True, case_sensitive=False)

# 2. Laden
kg_manager.load()

# 3. Analyse durchführen (Erkennt Calls & Assignments)
kg_manager.analyze_calls()

# 4. Speichern (überschreibt Standard, oder neuen Pfad angeben)
kg_manager.save(r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test_filled.ttl")

Das Verzeichnis 'd:\MA_Python_Agent' wurde zum sys.path hinzugefügt.
Lade Graphen: D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test_filled.ttl
Graph geladen: 1332 Tripel.
Starte POU Call Analyse (Case Sensitive: False)...
Analyse beendet. Erstellt: 27 Calls, 52 Assignments.
Speichere Graphen nach: D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test_filled.ttl


WindowsPath('D:/MA_Python_Agent/MSRGuard_Anpassung/KGs/Test_filled.ttl')